In [ ]:
# https://youtu.be/TibQO_xv1zc

#### Model Parallel

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# pseudocode
class ModelParallel(nn.Module):
    def __init__(self, ...):
        super(ModelParallel, self).__init__()
        self.part_1 = nn.Sequential(
            ...
        )
        self.part_2 = nn.Sequential(
            ...
        )
        # put each part on a different device
        
    def forward(self, x):
        x = x.to(torch.device('cuda:0'))
        x1 = self.part_1(x)
        # move to second device
        x1 = x1.to(torch.device('cuda:1'))
        y = self.part_2(x1)
        return y

In [ ]:
# regular training
for x, y_gt in data:
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y, y_gt)
    loss.backward()
    optimizer.step()

In [ ]:
# Gradient accumulation
for x, y_gt in data:
    optimizer.zero_grad()
    for sub_x, sub_y_gt in split(x, y_gt):
        sub_y_pred = model(sub_x)
        loss = criterion(sub_y_pred, sub_y_gt)
        loss.backward()
    optimzier.step()

#### Data Parallel

In [ ]:
# nn.DataParallel

model = MyModel()
# make it parallel
model = nn.DataParallel(model)

"""
1. Replicates model to all GPUs
2. Splits the batch to the GPUs
3. Sync updates
"""

In [ ]:
# DistirbutedDataParallel

def main():
    ...
    torch.multiprocessing.spawn(trian,
                                nproc=world_size,
                                args=args)

def train(rank, args):
    # init the process in context
    torch.distributed.init_process_group(backend='nccl', init_method='tcp://127.0.0.1:54263', world_size, rank)
    ...
    # wrap the model
    model = nn.parallel.DistributedDataParallel(model, device_ids=[rank])
    
    # use "distributed aware" sampler
    sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replica=world_size,
        rank=rank
    )
    
    loader = DataLoader(train_dataset, batch_size,
                        shuffle=False, sampler=train_sampler)


